<a href="https://colab.research.google.com/github/jasen8651/python001/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

예시 의사 결정나무 파일 및 https://wonhwa.tistory.com/35 링크
영화 스토리2 파일 참고/  https://western-sky.tistory.com/55
https://github.com/threegenie/sentiment_project
https://moojuksublime.tistory.com/14

In [ ]:
## 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

#  경로 변경
%cd /content/drive/MyDrive/python_demo

Mounted at /content/drive
/content/drive/MyDrive/python_demo


In [ ]:
!pip install konlpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 36.4 MB/s eta 0:00:00


In [ ]:
# 네이버 영화 리뷰 감성 분류하기
# 다음의 강좌를 거의 그대로 작성한 것임.
# https://wikidocs.net/44249

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [ ]:
# 네이버 평점 데이터 다운로드 링크
# https://github.com/e9t/nsmc/

# 1. 데이터 로드하기
train_data = pd.read_table('/content/drive/MyDrive/python_demo/ratings_train.txt')

test_data = pd.read_table('/content/drive/MyDrive/python_demo/ratings_test.txt', encoding='cp949')

print("훈련용 리뷰 개수: ", len(train_data))
print(train_data[:5])

print("테스트용 리뷰 개수: ", len(test_data))
print(test_data[:5])

# 2. 전처리 (토큰화 이전)

print(train_data['document'].nunique())
print(train_data['label'].nunique())
 
train_data.drop_duplicates(subset=['document'], inplace=True)

print('총 샘플의 수: ', len(train_data))



훈련용 리뷰 개수:  118164
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리    0.0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나    1.0
2  10265843                                  너무재밓었다그래서보는것을추천한다    0.0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정    0.0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...    1.0
테스트용 리뷰 개수:  50000
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
115393
2
총 샘플의 수:  115394


In [ ]:
train_data['label'].value_counts()#.plot(kind = 'bar')
# print(train_data.groupby('label').size())

# 리뷰 중에 NULL값을 가지는 샘플이 있는지...
print(train_data.isnull().values.any())

train_data.isnull().sum()

# NULL 값 샘플의 위치

print(train_data.loc[train_data.document.isnull()])

# NULL값 가진 샘플 제거

train_data = train_data.dropna(how = 'any')

print(train_data.isnull().values.any())

print(len(train_data))


True
            id document  label
25857  2172111      NaN    1.0
False
115392


In [ ]:

train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣㄱ-힣 ]", "")
print(train_data[:10])

train_data['document'] = train_data['document'].str.replace('^ +', "") #처음부터 여러개 공백이 나온다면 empty로 변경

train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

print(train_data.loc[train_data.document.isnull()][:5])

train_data = train_data.dropna(how = 'any')
print(len(train_data))



<ipython-input-7-784b8821a4b2>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣㄱ-힣 ]", "")


         id                                           document  label
0   9976970                                  아 더빙 진짜 짜증나네요 목소리    0.0
1   3819312                         흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나    1.0
2  10265843                                  너무재밓었다그래서보는것을추천한다    0.0
3   9045019                          교도소 이야기구먼 솔직히 재미는 없다평점 조정    0.0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...    1.0
5   5403919              막 걸음마 뗀 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움    0.0
6   7797314                               원작의 긴장감을 제대로 살려내지못했다    0.0
7   9443947  별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫...    0.0
8   7156791                             액션이 없는데도 재미 있는 몇안되는 영화    1.0
9   5912145          왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나    1.0
id            0
document    629
label         0
dtype: int64
           id document  label
404   4221289      NaN    0.0
412   9509970      NaN    1.0
470  10147571      NaN    1.0
584   7117896      NaN    0.0
593   6478189      

<ipython-input-7-784b8821a4b2>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "") #처음부터 여러개 공백이 나온다면 empty로 변경


In [ ]:
# 테스트 데이터에 대해 전처리

test_data.drop_duplicates(subset = ['document'], inplace=True)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
test_data['document'] = test_data['document'].str.replace('^ +', "")
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')

print('전처리 후 테스트용 샘플의 개수: ', len(test_data))


<ipython-input-8-631cf3fa766c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
<ipython-input-8-631cf3fa766c>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "")


전처리 후 테스트용 샘플의 개수:  48852


In [ ]:

# 3.토큰화
stopwords = ['의', '가', '이', '은', '들', '는','좀', '잘', '걍', '과', '도', '를', '을', '으로', '자', '에', '와', '한', '하다', '랑']

okt = Okt()
s = okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem=True)
print(s)


['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']


In [ ]:

x_train = []
for s in train_data['document']:
    temp_x = okt.morphs(s, stem=True)
    temp_x = [word for word in temp_x if not word in stopwords]
    x_train.append(temp_x)
    
print(x_train[:3])

x_test = []
for s in test_data['document']:
    temp_x = okt.morphs(s, stem=True)
    temp_x = [word for word in temp_x if not word in stopwords]
    x_test.append(temp_x)
    
print(x_test[:3])
 


KeyboardInterrupt: ignored

In [ ]:
# 4. 정수 인코딩. 단어집합 생성 및 각 단어에 고유한 정수 부여
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
print(tokenizer.word_index)

threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

# 단어와 빈도수의 쌍을 key와 value로 받는다
for key, value in tokenizer.word_counts.items():
    total_freq += value
    
    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt += 1
        rare_freq += value
    
print('단어 집합의 크기: ', total_cnt)
print('등장 빈도가 %d번 이하인 희귀 단어의 수: %d' % (threshold-1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율: ", (rare_cnt / total_cnt) * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율: ", (rare_freq / total_freq) * 100)

# 전체 단어 개수중 빈도수 2이하인 단어는 제거
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print("단어 집합의 크기: ", vocab_size)



{'영화': 1, '보다': 2, '없다': 3, '이다': 4, '있다': 5, '좋다': 6, '너무': 7, '다': 8, '정말': 9, '재밌다': 10, '되다': 11, '만': 12, '적': 13, '같다': 14, '진짜': 15, '로': 16, '않다': 17, '아니다': 18, '점': 19, '에서': 20, '만들다': 21, '평점': 22, '연기': 23, '나오다': 24, '것': 25, '나': 26, '최고': 27, '내': 28, '안': 29, '그': 30, '인': 31, '왜': 32, '못': 33, '스토리': 34, '생각': 35, '드라마': 36, '사람': 37, '게': 38, '감동': 39, '보고': 40, '이렇다': 41, '말': 42, '고': 43, '더': 44, '때': 45, '아깝다': 46, 'ㅋㅋ': 47, '감독': 48, '아': 49, '그냥': 50, '배우': 51, '재미': 52, '거': 53, '재미있다': 54, '요': 55, '하고': 56, '까지': 57, '시간': 58, '내용': 59, '중': 60, '뭐': 61, '지루하다': 62, '자다': 63, '재미없다': 64, '주다': 65, '네': 66, '쓰레기': 67, '수': 68, '모르다': 69, '가다': 70, '싶다': 71, '들다': 72, '알다': 73, '사랑': 74, '작품': 75, '지': 76, '하나': 77, '볼': 78, '그렇다': 79, '다시': 80, '정도': 81, '이건': 82, '마지막': 83, '저': 84, 'ㅋ': 85, '이렇게': 86, '액션': 87, 'ㅠㅠ': 88, 'ㅋㅋㅋ': 89, '완전': 90, '걸': 91, '차다': 92, '오다': 93, '많다': 94, '처음': 95, '돈': 96, '최악': 97, '개': 98, '장면': 99, '안되다': 100, '주인공': 101, '나다': 

In [ ]:
# 케라스 토크나이저로 넘겨주면, 텍스트 시퀀스를 정수 시퀀스로 변환해준다.

tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

print(x_train[:3])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

# 빈(empty) 샘플 제거

drop_train = [index for index, s in enumerate(x_train) if len(s) < 1]

x_train = np.delete(x_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(x_train))
print(len(y_train))



[[49, 455, 15, 252, 695], [1033, 423, 40, 615, 1, 190, 1569, 23, 979, 729, 17], [356, 2094, 2849, 3697, 2, 221, 8]]
34001
114663


/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [ ]:
# 5. 패딩

my_max = max(len(l) for l in x_train)
my_mean =  sum(map(len, x_train)) / len(x_train)  
print('리뷰의 최대 길이: ', my_max)
print('리뷰의 평균 길이: ', my_mean)

# plt.hist([len(s) for s in x_train], bins=50)
# plt.xlabel('length of samples')
# plt.ylabel('number of samples')
# plt.show()

max_len = 30
cnt = 0
for s in x_train:
    if len(s) <= max_len:
       cnt += 1
print('전체 심플 중 길이가 %d 이하인 샘플의 비율: %d%%'%(max_len, (cnt / len(x_train)) * 100))

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
       

# 6. LSTM으로 네이버 영화 리뷰 감성 분류하기

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)
    
loaded_model = load_model('best_model.h5')
print('\n 테스트 정확도: %.4f'%(loaded_model.evaluate(x_test, y_test)[1]))
    


리뷰의 최대 길이:  63
리뷰의 평균 길이:  10.41113496661863
전체 심플 중 길이가 30 이하인 샘플의 비율: 94%
Epoch 1/15
453/454 [============================>.] - ETA: 0s - loss: 0.4885 - acc: 0.7570
Epoch 1: val_acc improved from -inf to 0.81870, saving model to best_model.h5
454/454 [==============================] - 49s 97ms/step - loss: 0.4884 - acc: 0.7571 - val_loss: 0.4009 - val_acc: 0.8187
Epoch 2/15
453/454 [============================>.] - ETA: 0s - loss: 0.3627 - acc: 0.8417
Epoch 2: val_acc did not improve from 0.81870
454/454 [==============================] - 42s 92ms/step - loss: 0.3626 - acc: 0.8417 - val_loss: 0.4112 - val_acc: 0.8166
Epoch 3/15
453/454 [============================>.] - ETA: 0s - loss: 0.3306 - acc: 0.8597
Epoch 3: val_acc improved from 0.81870 to 0.82429, saving model to best_model.h5
454/454 [==============================] - 43s 95ms/step - loss: 0.3307 - acc: 0.8597 - val_loss: 0.3891 - val_acc: 0.8243
Epoch 4/15
453/454 [============================>.] - ETA: 0s - loss: 0.3101 

ValueError: ignored

In [ ]:
## 7. 리뷰 예측해 보기

def sentiment_predict(s):
    s = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", s)
    s = s.replace('^ +', "")
    
    s = okt.morphs(s, stem=True) # 토큰화
    s = [word for word in s if not word in stopwords] # 불용어 제거
        
    encoded = tokenizer.texts_to_sequences([s]) # 정수 인코딩
    padded = pad_sequences(encoded, maxlen=max_len) # 패딩
    score = float(loaded_model.predict(padded))
    if (score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

# text = input("영화 리뷰를 입력하세요: ")
# sentiment_predict(text)

sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')
sentiment_predict("이 영화 핵노잼 ㅠㅠ")
sentiment_predict("이딴게 영화냐 ㅉㅉ")
sentiment_predict("감독 뭐하는 놈이냐?")
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')